# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921690495026                   -0.69    5.9         
  2   -7.926162430884       -2.35       -1.22    1.0    265ms
  3   -7.926836729871       -3.17       -2.37    1.9    334ms
  4   -7.926861484121       -4.61       -3.03    2.5    302ms
  5   -7.926861639762       -6.81       -3.39    1.8    266ms
  6   -7.926861669345       -7.53       -3.78    1.9    270ms
  7   -7.926861680356       -7.96       -4.32    1.4    281ms
  8   -7.926861681709       -8.87       -4.75    2.2    286ms
  9   -7.926861681865       -9.81       -5.46    1.9    270ms
 10   -7.926861681872      -11.19       -5.88    2.0    305ms
 11   -7.926861681873      -12.05       -6.52    1.5    253ms
 12   -7.926861681873      -13.09       -7.21    2.2    301ms
 13   -7.926861681873      -14.57       -8.02    2.4    302ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921714253424                   -0.69    5.8         
  2   -7.926168658977       -2.35       -1.22    1.0    329ms
  3   -7.926837152617       -3.17       -2.37    1.8    292ms
  4   -7.926861520531       -4.61       -3.02    2.8    313ms
  5   -7.926861639638       -6.92       -3.37    2.0    277ms
  6   -7.926861668867       -7.53       -3.76    1.8    308ms
  7   -7.926861680563       -7.93       -4.36    1.4    247ms
  8   -7.926861681774       -8.92       -4.88    2.0    275ms
  9   -7.926861681861      -10.06       -5.27    2.0    274ms
 10   -7.926861681871      -11.00       -5.77    1.8    296ms
 11   -7.926861681873      -11.78       -7.15    1.8    269ms
 12   -7.926861681873      -13.20       -7.41    3.5    351ms
 13   -7.926861681873   +    -Inf       -8.33    1.0    254ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921696623428                   -0.69    5.9         
  2   -7.926170055836       -2.35       -1.22    1.0    268ms
  3   -7.926840296984       -3.17       -2.37    1.9    307ms
  4   -7.926864922539       -4.61       -3.00    2.6    328ms
  5   -7.926865038532       -6.94       -3.32    1.6    305ms
  6   -7.926865077076       -7.41       -3.71    1.4    257ms
  7   -7.926865091766       -7.83       -4.41    1.6    266ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
